In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def chrome_setting():
    chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless') # 크롬 탭이 나타나지 않게 옵션 조정
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage') # 공유 메모리를 담당하는 디렉토리 사용 x
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [18]:
# 웹드라이버 오픈
driver = chrome_setting()
driver.implicitly_wait(10)

# 장르 - 발라드 - 인기순 url
# 장르, 인기순, 인덱스 넘버 확인
url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
lyrics_list = []
singer_list = []
date_list = []
like_list = []
genre_list = []

# range값 지정
# 15000개 크롤링
for x in range(1,31):
    for y in range(1,11):
        for z in range(1, 51):
            time.sleep(3)
            click_element3 = WebDriverWait(driver, 2) \
                .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{z}]/td[4]/div/a')))
            click_element3.click()

            try:
                # 제목
                title = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))

                # 가사
                lyrics = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
                
                # 가수
                singer = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
                
                # 발매일
                date = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)")))
                
                # 좋아요 수
                like = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
                
                # 장르
                genre = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)")))
                
                title_list.append(title.text)
                lyrics_list.append(lyrics.text)
                singer_list.append(singer.text)
                date_list.append(date.text)
                like_list.append(like.text)
                genre_list.append(genre.text)

            except:
                print(x,y,z,"에서 에러 발생.")
            
            driver.back()
            
        time.sleep(1)
        click_element2 = WebDriverWait(driver, 2) \
            .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="pageObjNavgation"]/div/span/a[{y}]')))
        click_element2.click()

    time.sleep(1)
    
    # 이 코드는 인덱스 501이상부터 크롤링 할 때 ######
    # a_tag = 'a[2]'
    ####################################

    # 이 코드는 처음부터 크롤링 할 때 #########
    a_tag = 'a' if x==1 else 'a[2]'
    ####################################

    click_element1 = WebDriverWait(driver, 2) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="pageObjNavgation"]/div/{a_tag}')))
    click_element1.click()

2 5 43 에서 에러 발생.
2 7 6 에서 에러 발생.
6 7 39 에서 에러 발생.
9 1 24 에서 에러 발생.
9 2 43 에서 에러 발생.
10 9 19 에서 에러 발생.
11 5 45 에서 에러 발생.
11 9 21 에서 에러 발생.
12 6 31 에서 에러 발생.
12 6 44 에서 에러 발생.
13 5 32 에서 에러 발생.
13 9 9 에서 에러 발생.
14 2 31 에서 에러 발생.
14 5 8 에서 에러 발생.
15 4 22 에서 에러 발생.
15 4 35 에서 에러 발생.
15 5 8 에서 에러 발생.
15 5 17 에서 에러 발생.
15 6 22 에서 에러 발생.
15 6 23 에서 에러 발생.
15 7 48 에서 에러 발생.
15 9 7 에서 에러 발생.
15 9 19 에서 에러 발생.
16 4 4 에서 에러 발생.
16 4 9 에서 에러 발생.
16 4 12 에서 에러 발생.
16 6 50 에서 에러 발생.
16 7 34 에서 에러 발생.
16 9 12 에서 에러 발생.
17 2 11 에서 에러 발생.
17 4 4 에서 에러 발생.
17 4 18 에서 에러 발생.
17 4 46 에서 에러 발생.
17 5 10 에서 에러 발생.
17 7 37 에서 에러 발생.
17 8 10 에서 에러 발생.
17 9 12 에서 에러 발생.
18 3 13 에서 에러 발생.
18 5 18 에서 에러 발생.
18 6 20 에서 에러 발생.
18 7 3 에서 에러 발생.
18 7 14 에서 에러 발생.
18 8 37 에서 에러 발생.
18 8 48 에서 에러 발생.
18 9 24 에서 에러 발생.
19 2 38 에서 에러 발생.
19 6 22 에서 에러 발생.
19 7 2 에서 에러 발생.
19 8 19 에서 에러 발생.
20 1 28 에서 에러 발생.
20 8 13 에서 에러 발생.
20 9 46 에서 에러 발생.
21 4 22 에서 에러 발생.
21 8 3 에서 에러 발생.
22 4 29 에서 에러 발생.
23 1 9 에서 에러 발생.
23 1 33 에

In [19]:
raw_result = {'제목': title_list,
              '가사': lyrics_list,
              '가수' : singer_list,
              '발매일' : date_list,
              '좋아요수' : like_list,
              '장르': genre_list}

result = pd.DataFrame(raw_result)

In [24]:
print(result.shape)
result.tail(10)

(13346, 6)


,제목,가사,가수,발매일,좋아요수,장르
13336,나비가 날았습니다,왜 그렇게도 그대 지쳐 있죠\n모두 끝난 것처럼\n그 가슴 깊이 숨겨 둔\n얘기를 ...,김민석 (멜로망스),2021.08.24,362,"발라드, 국내드라마"
13337,Holiday,발 디딜 곳 없는 방\n숨 쉴 곳은 창문 틈\n전구 불빛에 날리는 먼지뿐\n아주 아...,제휘,2021.10.30,316,"발라드, 국내드라마"
13338,내가 한 걸음 뒤로 갈게,마치 꿈처럼 내 곁에 다가온\n햇살 가득한 너의 그 웃음을\n조금 느리게 느리게 걸...,전상근,2022.01.18,198,"발라드, 국내드라마"
13339,그대에게 가까이,점점 그대에게 가까이\n조금 더 가까이\n나도 몰래 마음이\n그대에게 마음이\n아무...,단칸방 로맨스,2022.02.21,552,발라드
13340,A의 테마 (Feat. 이태권),너의 모든 것은\n내겐 첨 겪었던 설레임\n살짝 풍기던 그 향수 냄새도\n곧게 걷던...,015B,2022.04.13,415,발라드
13341,긴 밤을 나 혼자 보내기엔,나는 알지 못했죠\n그대 내게 얼마나\n소중했었는지\n이제 알 것 같아요\n그땐 내...,최도영(CANDO),2022.06.26,154,"발라드, 국내드라마"
13342,Rooting for you,오 기억하나요 설레였나요\n그 여름 함께였던 우리를\n그시간 속에서 사계절의 우리를...,DIA (다이아),2022.09.14,641,발라드
13343,지금 청혼하는 거야,지금 네게 청혼을 하는 거야\n미친 듯이 뛰는\n심장을 부여잡고\n날씨마저\n화창한...,이성혁,2022.10.08,64,"발라드, 인디음악"
13344,My First Love (나와 결혼해 줄래요) (inst.),그대 눈을 바라볼 때면\n느낄 수 있죠\n떨리는 맘 um\n말하지 않아도 내 맘이\...,허각,2022.10.16,797,발라드
13345,매직카펫,별빛조차 잠들은 밤에\n회색빛 네 숨소리를 들어\n어둠 속에서 혼자 뒤척이는 널\n...,스웨덴세탁소,2022.10.26,174,"발라드, 인디음악"


In [25]:
# csv 파일로 save
# result.to_csv("data/melon_ballad_1_15000.csv", index=False)